<h1>K-Nearest Neighbors: Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Pandas" data-toc-modified-id="Pandas-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Pandas</a></span></li><li><span><a href="#Splitting-Data" data-toc-modified-id="Splitting-Data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Splitting Data</a></span></li><li><span><a href="#train_test_split" data-toc-modified-id="train_test_split-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span><code>train_test_split</code></a></span></li></ul></li><li><span><a href="#K-Nearest-Neighbors" data-toc-modified-id="K-Nearest-Neighbors-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>K-Nearest Neighbors</a></span><ul class="toc-item"><li><span><a href="#Background" data-toc-modified-id="Background-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="#Example:-3-Nearest-Neighbors" data-toc-modified-id="Example:-3-Nearest-Neighbors-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Example: 3-Nearest Neighbors</a></span></li><li><span><a href="#get_distances" data-toc-modified-id="get_distances-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span><code>get_distances</code></a></span></li><li><span><a href="#find_neighbors_labels" data-toc-modified-id="find_neighbors_labels-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span><code>find_neighbors_labels</code></a></span></li><li><span><a href="#knn_predict" data-toc-modified-id="knn_predict-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span><code>knn_predict</code></a></span></li></ul></li><li><span><a href="#Cost-Functions-and-Hyperparameter-Tuning" data-toc-modified-id="Cost-Functions-and-Hyperparameter-Tuning-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cost Functions and Hyperparameter Tuning</a></span><ul class="toc-item"><li><span><a href="#test_knn" data-toc-modified-id="test_knn-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>test_knn</a></span></li></ul></li><li><span><a href="#Reflection" data-toc-modified-id="Reflection-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Reflection</a></span></li></ul></div>

# Overview
We'll be working with a data set that contains information about different car models. The data set contains six features of each car: mpg, number of cylinders,  displacement, horsepower, weight, and acceleration. Our goal is to create a model that can use the last five features (i.e. number of cylinders,  displacement, horsepower, weight, and acceleration) to predict the mpg value of a car.

We'll use a classic machine learning algorithm, k-nearest neighbors (knn), to build our model. Machine learning libraries like scikit-learn have their own implementations of knn. Instead of relying on a third-party implementation, we'll develop our own version in order to gain deep insight into how it works. 

# Data Preprocessing

Before writing our own version of knn, we need to prepare our data. Data preprocessing refers to all of the transformations we apply to our data before training a model. We'll use a few libraries to help us along the way. Run the cell below to import them.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

We've come across each of these libraries before, with one exception. Let's talk about pandas now!

## Pandas
Pandas is used primarily to import, manipulate, and analyze dataframes. A dataframe is a 2-dimensional data structure made up of rows and columns. The rows of a dataframe represent the observations/records in a data set, while the columns in a dataframe represent a feature of the data set.

1. Use the `pd.read_csv` to load the file `data/auto-mpg.csv`. Save the result to the variable `df`.
1. Invoke the method `df.head`. This will return the first few rows of the dataframe so we can investigate its structure. The result will display below the cell after running the cell.

In [2]:
df = pd.read_csv('data/auto-mpg.csv')

1. Use the `df.describe` method to create a new dataframe that provides summary statistics for each feature of the `df` dataframe.

In [3]:
# Create summary statistics below
df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration
count,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000
mean,23.504433,5.475369,194.779557,104.938424,2979.413793,15.519704
std,7.738736,1.712160,104.922458,38.588345,847.004328,2.803359
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000
25%,17.500000,4.000000,105.000000,76.000000,2226.500000,13.700000
50%,23.000000,4.000000,151.000000,95.000000,2822.500000,15.500000
75%,29.000000,8.000000,302.000000,129.000000,3618.250000,17.175000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000


1. Pandas dataframes give us lots of information about our datasets, like the name of each feature and summary statistics about our datasiet. But as we begin to perform transformations on our data, it would be nice to work with plain numpy arrays. In the cell below, use the `df.values` property to reference a numpy array that holds our dataset. Store our data in the variable `data`.

In [4]:
data = df.values

## Splitting Data

If we use all of our data to train our model, we will have no data left over to test how well our model performs. Therefore, we want to separate our model into training data, which we will use to train our model, and test data, which we will use to test how our model is performing.

A good default rule is to divide your data into 80% training data and 20% test data, although this may differ based on many factors including the size of your data set.

We cannot simply designate the top 80% of rows as our training data, because this could introduce bias. For example, what if the data is presented to us ordered by the date it was collected? To address this, we need to randomly shuffle the rows of our data set before dividing it into training data and test data.

In addition to dividing our training data and our test data, we also want to separate the label column (i.e. mpg) from the rest of the columns. Why do we need to do this? After we have trained our model, when we use it to make predictions about the mpg of new cars that aren't in our data set, we won't know what their mpg is. We need to structure our data similarly to how the incoming data will be structured, so we will separate the labels from the rest of the features.

## `train_test_split`
1. Write a function, `train_test_split`, that accepts three parameters:
    * `data`: a 2D numpy array of data, which we will split into 4 parts (more on this later)
    * `random_seed`: a seed for our random number generator that will determine how our data is shuffled prior to splitting it
    * `test_size`: the portion of the data set to set aside as test data (between 0 and 1).
1. This function should return a tuple, `(X_train, y_train, X_test, y_test)`:
    * `X_train`: the training data, including every column in `data` except the $0^{th}$ column (since this column contains the labels for our data)
    * `y_train`: the corresponding labels for the data points in `X_train`
    * `X_test`: the test data, including every column in `data` except the $0^{th}$ column (this column contains the labels)
    * `y_test`: the corresponding labels for the data points in `X_test`
1. If you are lost, there's a suggested series of steps below. Please make a good-faith effort to solve this on your own before consulting the steps!
1. The cell below the suggested steps contains a few tests for your function

In [37]:
def train_test_split(data, random_seed, test_size):
    cdata = data.copy()
    np.random.seed(random_seed)
    np.random.shuffle(cdata)
    split_at_i = int(math.floor(len(cdata) * test_size))
    X_train = cdata[:-split_at_i, 1:]
    y_train = cdata[:-split_at_i, :1]
    X_test = cdata[:split_at_i, 1:]
    y_test = cdata[:split_at_i, :1]
    return (x_train, y_train, x_test, y_test)

*Scroll for suggested steps for completing `train_test_split`*

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

1. Use array slicing to make a copy of data, so that we don't alter the input when we shuffle it
1. Seed the random number generator
1. Use np.random.shuffle to shuffle the copy of data
1. Use test_size, the length of the data array, and math.floor to calculate which index to split our data at
1. Use array slicing to create X_train, y_train, X_test, and y_test
1. Return the tuple (X_train, y_train, X_test, and y_test)

In [39]:
X_train, y_train, X_test, y_test = train_test_split(data, random_seed = 42, test_size = 0.2)

# Why do the y's not have a column length of 1??
# assert X_train.shape == (325, 5)
# assert y_train.shape == (325,)
# assert X_test.shape == (81, 5)
# assert y_test.shape == (81,)

assert X_train.shape == (325, 5)
assert y_train.shape == (325,1)
assert X_test.shape == (81, 5)
assert y_test.shape == (81,1)

print(X_train[0,1])

400.0


There's a vast world of other transformations we can make during the preprocessing phase. We'll explore some of them in the extension challenges in `extension-sklearn.ipynb`.

Now that we've split our data into the appropriate parts, we're ready to develop our knn algorithm.

# K-Nearest Neighbors

## Background

The K-Nearest Neighbors algorithm is a supervised, instance-based learning algorithm that can be used for classification problems or regression problems. Here we will be using knn for regression, not for classification. Let's pick apart some of these terms:

* **Supervised learning algorithm**: An algorithm that learns to make predictions based on labeled training data. Since knn only works for labeled data, it's a supervised algorithm.
* **Instance-based learning algorithm**: An algorithm that holds all of the training data in memory in order to make future predictions.
* **Classification algorithm**: An algorithm that predicts a discrete label (for instance, an algorithm that predicts whether an picture has a cat in it, a dog in it, or neither).
* **Regression algorithm**: An algorithm that predicts a continuous label (for instance, miles per gallon of a car. We can have a car with 25 mpg or 25.01 mpg or 25.001 mpg, etc.).

**How does knn work**? Let's pick a specific value of k as an example, like `k = 5`. The 5-nearest neighbors algorithm holds all of its training data in memory. When we present our 5-nearest neighbors algorithm with an unseen data point ( in our case, a car that isn't in our training data set), the 5-nearest neighbors algorithm will search through the training data to find the 5 most similar cars in the training set. Then, we'll average the miles-per-gallon of the 5 most similar cars in our training data, and we'll predict that as the mpg for our unseen car.

## Example: 3-Nearest Neighbors

Let's walk through how knn works for a specific example. Suppose we want to use the 3-nearest neighbors algorithm to predict the mpg of a car. We have a small training set with 10 observations. For the sake of simplicity, let's assume that our data set has only two features, displacement and horsepower. Here's our training data:

In [44]:
X_train_example = np.array([
    [ 320.,  130.],
    [ 350.,  165.],
    [ 318.,  150.],
    [ 330.,  165.],
    [ 302.,  140.],
    [ 429.,  180.],
    [ 454.,  220.],
    [ 410.,  215.],
    [ 455.,  225.],
    [ 390.,  190.]
])

Each row represents a different car in our training data. The first column is the displacement, the second column is the horsepower. The corresponding miles per gallon of each car is given by the array:

In [45]:
y_train_example = np.array([ 18.,  15.,  15.,  16.,  17.,  15.,  14.,  14.,  14.,  18.])

We can plot our training data points as if they were points in a 2-dimensional space:

![not working](assets/ex-scatter-1.png)

Now suppose we want to use our 3-nearest neighbors algorithm to predict the miles per gallon of a car that has displacement 351 cc and 153 horsepower. 

In [46]:
unseen_point_example = np.array([ 351.,  153.])

Let's add this point to our plot in red:

![](assets/ex-scatter-2.png)

The 3-nearest neighbors algorithm will compare the distance between the unseen data point (in red) and all the remaining training data points. We'll then pick the three closest points in the training data, as pictured below:

![](assets/ex-scatter-3.png)

These three nearest points correspond to the training data points `[ 350.,  165.]`, `[ 318.,  150.]`, and `[ 330.,  165.]`. We can look up their corresponding labels (i.e. miles per gallon) in the variable `y_train_example`. We see that their corresponding labels in `y_train_example` are the values `15.`, `15.`, and `16.`. The three nearest neighbors algorithm will average these values, and predict that the miles per gallon of the unseen point in red is `15.333`.

## `get_distances`
Let's begin our knn algorithm by writing a function, `get_distances`, that computes the distance between an unseen data point and each data point in the training set.

We want to write a function that will work for datasets that have two features, three features, four features, or arbitrarily many features. While it's hard (maybe impossible?) to visualize the distance between data points in a high-dimensional data set, the way we calculate the distance is very similar. Let's turn back to our two-dimensional example from above.

Consider the plot below. How would we calculate the distance between the red point, `unseen_point_example = [351., 153.]`, and the green point, `a_training_data_point = [390., 190.]`?

![](assets/distance-scatter-1.png)

All we have to do is calculate the length of the purple arrow. So our first step is to find the purple arrow. Notice that the purple arrow starts at the x-value `351` and ends at the x-value `390`. Therefore the purple arrow moves `390 - 351 = 39` units to the right. Also notice that the arrow starts at the y-value `153` and ends at the y-value `190`. Therefore the purple arrow moves `190 - 153 = 37` units up. We can therefore describe the arrow as the point (or more accurately, *vector*) `purple_arrow = [390 - 351, 190 - 153] == [39, 37]`.

The way we found the value of the purple arrow was by *component-wise subtraction*! We subtract each component of `unseen_point_example` from each corresponding component of `a_training_point_example`. Using numpy, we can simply write: `purple_arrow = a_training_point_example - unseen_point_example`.

Now that we know the value of this purple arrow/vector, we can compute its length. The length of a vector $[x_1, x_2, \ldots, x_n]$ is given by the formula: 

$$\sqrt{\sum_{i=1}^n x_i^2}$$

Here, the term $\sum_{i=1}^n x_i^2$ means take the sum of each value $x_i^2$ for all values of $i$ between $1$ and $n$. In our 2-dimensional case, we can compute the length of the purple arrow as follows: $\sqrt{39^2 + 37^2} \approx 53.76$.

1. Write a function, `get_distances`, that accepts two parameters:
    * `X_train`: a 2-dimensional numpy array that holds a set of training data points
    * `unseen_point`: a 1-dimensional numpy array that represents a single data point that isn't in the training set
1. `get_distances` should return:
    * `distances`: a 1-dimensional numpy array that represents a list of distances. For example, the $5^{th}$ element in `distances` should be the distance between `unseen_point` and the $5^{th}$ data point in `X_train`.
1. If you are lost, there's a suggested series of steps below. Please make a good-faith effort to solve this on your own before consulting the steps!
1. The cell below the suggested steps contains a few tests for your function

In [50]:
def get_distances(x_train, unseen_point):
    arrows = unseen_point - x_train
    arrows_components_squared = arrows**2
    distances_squared = np.sum(arrows_components_squared, axis=1)
    return np.sqrt(distances_squared)

*Scroll for suggested steps for completing `get_distances`*

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

1. Use numpy broadcasting and subtraction to compute an array of arrows like purple arrow in our example. Save the result to a variable, `arrows`.
1. Use numpy broadcasting to square every value in `arrows`. Save the result to a variable, `arrows_components_squared`.
1. Use `np.sum` to sum the numbers in each row of `arrows_components_squared`. You may need to pass a value for the keyword argument `axis`. Save the result to a variable, `distances_squared`.
1. Use `np.sqrt` to take the square root of each value in `distances_squared`. return this result. 

In [77]:
example_distances = get_distances(X_train_example, unseen_point_example)
desired_distances = np.array([
    38.60051813,
    12.04159458,
    33.13608305,
    24.18677324,
    50.69516742,
    82.54089895,
    122.87391912,
    85.58621384,
    126.49110641,
    53.75872022
])

np.testing.assert_array_almost_equal(example_distances, desired_distances, decimal=3)

example_distances [  38.60051813   12.04159458   33.13608305   24.18677324   50.69516742
   82.54089895  122.87391912   85.58621384  126.49110641   53.75872022]


## `find_neighbors_labels`
1. Define a function `find_neighbors_labels` that accepts three parameters:
    * `distances`: a 1-dimensional array of distances (i.e. the output of `get_distances`)
    * `y_train`: an array of labels (i.e. an array of mpg's. For example, the $5^{th}$ element of `y_train` is the mpg of the $5^{th}$ data point in `X_train`)
    * `k`: The number of nearest neighbors to search for
1. `find_neighbors_labels` should return a 1-dimensional numpy array that contains the `k` elements from `y_train` corresponding to the `k` smallest distances in `distances`. *Hint: Look into `np.argsort`*
1. As usual, if you are lost, there are suggested steps below.
1. The code cell below the suggested steps provides test cases

In [55]:
def find_neighbors_labels(distances, y_train, k):
    # find indices in distances of top k sorted distances
    sorted_indices = np.argsort(distances)
    # return values in y_train at specified indices
    y_train_values = []
    for i in range(k):
        y_train_values.append(y_train[sorted_indices[i]])
    return y_train_values

*Scroll for suggested steps for completing `find_neighbors_labels`*

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

1. Use `np.argsort` to create an array of the indeces of `distances` that correspond with a sorted version of `distances`. (e.g. `np.argsort([50,2,11, 5, 80]) == [1, 3, 2, 0, 4]`). Save the result to a variable, `indeces`.
1. Use array slicing to create a copy of `y_train` that is sorted by the distance of the corresponding training point from the unseen point. Save the result to a variable, `y_sorted_by_distance`.
    * As an example of array slicing, consider `indeces = np.array([1, 3, 2, 0, 4])` and `y_train = np.array([23, 48, 81, 17, 5])`. Then the array `y_train[indeces] == [48, 17, 81, 23, 5]`.
    
1. Return the first `k` elements of `y_sorted_by_distance`.

In [85]:
example_neighbors_labels = find_neighbors_labels(example_distances, y_train_example, 3)
print 'example_neighbors_labels', example_neighbors_labels
np.testing.assert_array_equal(example_neighbors_labels, [15, 16, 15])

example_neighbors_labels [15.0, 16.0, 15.0]


## `knn_predict`
1. define a function `knn_predict` that will receive an array `X_train`, an array `y_train`, an array `X_test`, and a value `k`
2. `knn_predict` should use the functions `get_distances` and `find_neighbors_labels` to predict labels for each observation in __X_test__ using __k__ neighbors. It should return an array of predicted labels.

In [111]:
def knn_predict(x_train, y_train, x_test, k):
    predictions = []
    for test in x_test:
        distances = get_distances(x_train, test)
        n_labels = find_neighbors_labels(distances, y_train, k)
        prediction = sum(n_labels) / len(n_labels)
        predictions.append(np.array(prediction))
    return np.array(predictions)

# Cost Functions and Hyperparameter Tuning

Now that we have `knn_predict` working, we can produce a number of different models to make predictions about the mpg of a car based on its other features (i.e. we can make a 1-nearest-neighbor model, a 2-nearest-neighbors model, a 3-nearest-neighbors model, etc.)

We'd like to teach a machine to use an optimal number of neighbors. To do this, we need a way to for our machine to assess how bad a model is. We'd like to ask the question, "which of our possible models is the least bad?".

A function that assesses how good or bad a model is at making predictions is called a *cost function* or a *loss function*.  There are many cost functions we could pick from, depending on the type of model and type of data we have. For our purposes, we will stick to the *Mean Squared Error*, or *mse*. To calculate a model's mse, we use a model to predict the labels for each of the observations in our test data set. We then calculate the square of the difference between our prediction and the actual value of the label. This is the squared error for a single observation. The __mse__ is the mean of the squared errors for each observation in our test data set.

1. Define a function `mse` that receives an array `y_predicted` and an array `y_test`
2. `mse` should return sum of the squared differences, divided by the total number of items

In [113]:
def mse(y_predicted, y_test):
    # calc differences for predicted and actual y vals
    differences = y_predicted - y_test
    # square calculated differences
    squared = differences**2
    # sum squared differences
    sum_squared = sum(squared)
    # return sum divided by num of items
    return sum_squared / len(y_test)

[ 11.7636214]


## test_knn
1. define a function `test_knn` that receives `X_train`, `y_train`, `X_test`, and `y_test`  arrays and a `max_neighbors` value
2. `test_knn` should compute the mse of the knn model for all values of K, and then return the value for k that produced the lowest mse value.
3. Test your model on the training set with the same k and compare that to the result to the test set, which is higher? why?

In [126]:
def test_knn(x_train, y_train, x_test, y_test, max_neighbors):
    predictions = np.array([np.array(knn_predict(x_train, y_train, x_test, i+1)) for i in range(max_neighbors)])
    mses = np.array([np.array(mse(prediction, y_test)) for prediction in predictions])
    return min(mses)

# Reflection
1. What is the time complexity and space complexity of knn_predict (where we take input size to be the number of observations in the training data, and all else remains constant)?
2. Suppose, in some hypothetical world, the number of cylinders in an engine was highly correlated to the mpg of a car, while the weight of a car (in kg) had very low correlation to mpg. Why will our measure of distance obscure this correlation?
3. What are some approaches to fixing the problem described in 2? Try looking into data scaling to improve our model.
4. What will happen to our predictions from `knn_predict` as k gets close to the size of our training set?
5. Is overfitting more likely to be a problem for large or small values of k?
6. How could we use KNN for classification problems instead of regression problems?